In [1]:
import requests
import numpy as np
import csv 
import pandas as pd

In [22]:
AZURE_SUBSCRIPTION_KEY = "3e8bfdcfae0c41569d5c54ec94e8503a"
AZURE_ENDPOINT = "https://westus2.api.cognitive.microsoft.com/text/analytics/v2.0/"

In [23]:
def run_azure(input_array):
        headers = { 'Ocp-Apim-Subscription-Key' : AZURE_SUBSCRIPTION_KEY }
        sentiment_api_url =  AZURE_ENDPOINT + "sentiment"
        docs = []
        assert(len(input_array.flatten()) < 1000)
        for i, input_text in enumerate(input_array.flatten()):
            docs.append({'id' : str(i+1), 'language' : 'en', 'text' : input_text})
        documents = { 'documents' : docs }
        response = requests.post(sentiment_api_url, headers=headers, json=documents)
        sentiments = response.json()
        print(sentiments)
        scores = [x['score'] for x in sentiments['documents']]
        scores = np.array(scores)
        return scores.reshape(input_array.shape)

In [24]:
inputs = ["this is a text", "this is another text"]
input_array = np.array(inputs)
result = run(input_array)
print(result)

{'documents': [{'score': 0.7778275609016418, 'id': '1'}, {'score': 0.7778275609016418, 'id': '2'}], 'errors': []}
[0.77782756 0.77782756]


In [34]:
def write_results(tweet_list, label_list, api_func, save_file): 
    with open(save_file, 'w') as f: 
        csv_writer = csv.writer(f)
        csv_writer.writerow(["text", "label", str(api_func)])
        sentiment = api_func(np.array(tweet_list))
        for i, twt in enumerate(tweet_list): 
            csv_writer.writerow([twt, label_list[i], sentiment[i]])

In [38]:
tweets = [] 
labels = [] 

with open('datasets/twitter_sentiment100.csv', 'r') as f: 
    csv_reader = csv.reader(f)
    row = next(csv_reader)
    for row in csv_reader: 
        tweets.append(row[0])
        labels.append(row[1])

write_results(tweets, labels, run_azure, 'results/twitter_sentiment100_results.csv')

{'documents': [{'score': 0.9631977081298828, 'id': '1'}, {'score': 0.8952233791351318, 'id': '2'}, {'score': 0.20380455255508423, 'id': '3'}, {'score': 0.16436517238616943, 'id': '4'}, {'score': 0.9923532009124756, 'id': '5'}, {'score': 0.7719802856445312, 'id': '6'}, {'score': 0.8556222915649414, 'id': '7'}, {'score': 0.9133977293968201, 'id': '8'}, {'score': 0.07152354717254639, 'id': '9'}, {'score': 0.7941385507583618, 'id': '10'}, {'score': 0.7449294328689575, 'id': '11'}, {'score': 0.11761602759361267, 'id': '12'}, {'score': 0.5, 'id': '13'}, {'score': 0.9036202430725098, 'id': '14'}, {'score': 0.9851231575012207, 'id': '15'}, {'score': 0.1201251745223999, 'id': '16'}, {'score': 0.8197458982467651, 'id': '17'}, {'score': 0.05911615490913391, 'id': '18'}, {'score': 0.8487657904624939, 'id': '19'}, {'score': 0.20968815684318542, 'id': '20'}, {'score': 0.5, 'id': '21'}, {'score': 0.5, 'id': '22'}, {'score': 0.8717667460441589, 'id': '23'}, {'score': 0.5, 'id': '24'}, {'score': 0.5, '

In [20]:
df = pd.read_csv('results/sst_sentiment100_results.csv')
print(df.mean()) 
print(df.sem())

label              0.410000
Azure Sentiment    0.570582
g_sentiment       -0.025000
g_magnitude        0.524000
dtype: float64
label              0.049431
Azure Sentiment    0.034112
g_sentiment        0.060275
g_magnitude        0.030686
dtype: float64


In [21]:
label = df['label'].tolist() 
pred = df['g_sentiment'].tolist()

In [22]:
label = np.asarray(label)
new_pred = [] 
for p in pred: 
    if float(p) > 0: 
        new_pred.append(1)
    else: 
        new_pred.append(0)
pred = np.asarray(new_pred)
pred = np.around(pred)
acc = np.sum(pred==label)
print("Accuracy", acc/len(label))

Accuracy 0.81
